In [1]:
import os
import tensorflow as tf
import numpy as np
import librosa
import soundfile as sf
import math
from tensorflow.keras import Model, layers, optimizers

In [2]:
# from google.colab import drive
# drive.mount('/content/drive')

In [3]:
# Paths to data

train_blended_paths = '../../data/audios/english/train/blended'
train_clean_paths = '../../data/audios/english/train/clean'

# train_blended_paths = '../../data/audios/english/train/blended_trim'
# train_clean_paths = '../../data/audios/english/train/clean_trim'

# train_blended_paths = '/content/drive/MyDrive/DAN_data/train/blended_trim'
# train_clean_paths = '/content/drive/MyDrive/DAN_data/train/clean_trim'

# ---------------------------------------------------------------------------

val_blended = '../../data/audios/english/validation/blended'
val_clean = '../../data/audios/english/validation/clean'

# val_blended = '/content/drive/MyDrive/DAN_data/validation/blended_trim'
# val_clean = '/content/drive/MyDrive/DAN_data/validation/clean_trim'

# ---------------------------------------------------------------------------

test_blended = '../../data/audios/english/test/blended'
test_clean = '../../data/audios/english/test/clean'

In [4]:
# Constants
SAMPLE_RATE = 16000  # Define sample rate for consistency
TARGET_LENGTH = SAMPLE_RATE * 5  # Set target length in samples (3 seconds here as an example)
batch_size = 32
reduction_factor = 75

In [5]:
# Функція для завантаження аудіо та приведення до потрібного формату
def load_audio_tf(path, target_sr=SAMPLE_RATE, target_length=TARGET_LENGTH):
    audio, sr = librosa.load(path, sr=target_sr)
    if len(audio) > target_length:
        audio = audio[:target_length]
    else:
        audio = np.pad(audio, (0, max(0, target_length - len(audio))))
    return audio

# Функція для отримання всіх файлів у директорії (оригінальна версія без обмеження)
def get_all_files(directory, extension=".mp3"):
    file_paths = {}
    for root, _, files in os.walk(directory):
        for file in files:
            if file.endswith(extension):
                file_name = os.path.splitext(file)[0]
                file_paths[file_name] = os.path.join(root, file)
    return file_paths

# Функція для отримання парних файлів з обмеженням розміру набору даних
def get_limited_files(directory, extension=".mp3", limit=None):
    file_paths = {}
    count = 0
    for root, _, files in os.walk(directory):
        for file in files:
            if file.endswith(extension):
                file_name = os.path.splitext(file)[0]
                file_paths[file_name] = os.path.join(root, file)
                count += 1
                if limit and count >= limit:
                    return file_paths
    return file_paths

# Функція для створення tf.data.Dataset із парних файлів
def audio_to_tf_dataset(blended_dir, clean_dir, batch_size=16, shuffle=True, reduction_factor=20):
    # Отримуємо обмежену кількість файлів у кожному наборі
    blended_files = get_limited_files(blended_dir, '.mp3', limit=None)
    clean_files = get_limited_files(clean_dir, '.flac', limit=None)

    # Створюємо пари файлів, які мають однакову назву
    common_files = list(blended_files.keys() & clean_files.keys())
    
    # Обмежуємо кількість парних файлів у залежності від reduction_factor
    limited_common_files = common_files[::reduction_factor]
    
    blended_paths = [blended_files[name] for name in limited_common_files]
    clean_paths = [clean_files[name] for name in limited_common_files]
    
    # Створюємо генератор для пар
    def generator():
        for b_path, c_path in zip(blended_paths, clean_paths):
            blended_audio = load_audio_tf(b_path)
            clean_audio = load_audio_tf(c_path)
            yield blended_audio, clean_audio
    
    dataset = tf.data.Dataset.from_generator(
        generator,
        output_signature=(
            tf.TensorSpec(shape=(TARGET_LENGTH,), dtype=tf.float32),
            tf.TensorSpec(shape=(TARGET_LENGTH,), dtype=tf.float32),
        )
    )
    if shuffle:
        dataset = dataset.shuffle(buffer_size=1000)
    dataset = dataset.batch(batch_size)
    return dataset

In [6]:
def optimized_unet(input_shape=(None, 1)):
    inputs = layers.Input(shape=input_shape)
    
    conv1 = layers.Conv1D(64, kernel_size=3, activation='relu', padding='same')(inputs)
    conv1 = layers.Conv1D(64, kernel_size=3, activation='relu', padding='same')(conv1)
    pool1 = layers.MaxPooling1D(pool_size=2)(conv1)

    conv2 = layers.Conv1D(128, kernel_size=3, activation='relu', padding='same')(pool1)
    conv2 = layers.Conv1D(128, kernel_size=3, activation='relu', padding='same')(conv2)
    pool2 = layers.MaxPooling1D(pool_size=2)(conv2)

    conv3 = layers.Conv1D(256, kernel_size=3, activation='relu', padding='same')(pool2)
    conv3 = layers.Conv1D(256, kernel_size=3, activation='relu', padding='same')(conv3)

    up4 = layers.UpSampling1D(size=2)(conv3)
    up4 = layers.concatenate([up4, conv2])
    conv4 = layers.Conv1D(128, kernel_size=3, activation='relu', padding='same')(up4)
    conv4 = layers.Conv1D(128, kernel_size=3, activation='relu', padding='same')(conv4)

    up5 = layers.UpSampling1D(size=2)(conv4)
    up5 = layers.concatenate([up5, conv1])
    conv5 = layers.Conv1D(64, kernel_size=3, activation='relu', padding='same')(up5)
    conv5 = layers.Conv1D(64, kernel_size=3, activation='relu', padding='same')(conv5)

    outputs = layers.Conv1D(1, kernel_size=1, activation='linear')(conv5)

    model = Model(inputs, outputs)
    return model

In [7]:
# Ініціалізація та компіляція моделі
model = optimized_unet(input_shape=(TARGET_LENGTH, 1))
model.compile(optimizer=optimizers.Adam(learning_rate=1e-4), loss='huber', metrics=['mae'])

In [8]:
train_dataset = audio_to_tf_dataset(train_blended_paths, train_clean_paths, batch_size=batch_size, reduction_factor=reduction_factor).repeat()
val_dataset = audio_to_tf_dataset(val_blended, val_clean, batch_size=batch_size, reduction_factor=reduction_factor).repeat()


In [9]:
# Підрахунок кількості парних файлів у повному навчальному та валідаційному наборах
num_train_samples = len(get_all_files(train_blended_paths, ".mp3").keys() & get_all_files(train_clean_paths, ".flac").keys())
num_val_samples = len(get_all_files(val_blended, ".mp3").keys() & get_all_files(val_clean, ".flac").keys())

# Зменшуємо кількість зразків за допомогою reduction_factor
num_train_samples = num_train_samples // reduction_factor
num_val_samples = num_val_samples // reduction_factor

# Обчислення steps_per_epoch та validation_steps
steps_per_epoch = max(1, num_train_samples // batch_size)
validation_steps = max(1, num_val_samples // batch_size)

In [10]:
# Підрахунок кількості кроків
# num_train_samples = len(train_dataset) * reduction_factor
# num_val_samples = len(val_dataset) * reduction_factor

steps_per_epoch = max(1, num_train_samples // batch_size)
validation_steps = max(1, num_val_samples // batch_size)

In [11]:
print("Кількість пар у тренувальному наборі:", num_train_samples)
print("Кількість пар у валідаційному наборі:", num_val_samples)
print("Кроки на епоху:", steps_per_epoch)
print("Кроки на валідацію:", validation_steps)

Кількість пар у тренувальному наборі: 380
Кількість пар у валідаційному наборі: 35
Кроки на епоху: 11
Кроки на валідацію: 1


In [12]:
# Навчання моделі
EPOCHS = 5
history = model.fit(
    train_dataset,
    validation_data=val_dataset,
    epochs=EPOCHS,
    steps_per_epoch=steps_per_epoch,
    validation_steps=validation_steps
)

Epoch 1/5
11/11 ━━━━━━━━━━━━━━━━━━━━ 3432s 309s/step - loss: 0.0018 - mae: 0.0330 - val_loss: 0.0027 - val_mae: 0.0367
Epoch 2/5
11/11 ━━━━━━━━━━━━━━━━━━━━ 3518s 324s/step - loss: 0.0014 - mae: 0.0296 - val_loss: 0.0014 - val_mae: 0.0242
Epoch 3/5
11/11 ━━━━━━━━━━━━━━━━━━━━ 3435s 311s/step - loss: 9.5393e-04 - mae: 0.0267 - val_loss: 0.0014 - val_mae: 0.0277
Epoch 4/5
11/11 ━━━━━━━━━━━━━━━━━━━━ 3453s 314s/step - loss: 8.1585e-04 - mae: 0.0250 - val_loss: 5.7297e-04 - val_mae: 0.0196
Epoch 5/5
11/11 ━━━━━━━━━━━━━━━━━━━━ 3474s 317s/step - loss: 8.5932e-04 - mae: 0.0255 - val_loss: 0.0011 - val_mae: 0.0254


In [13]:
# Evaluate and save the model
model.save("ML-DAN_v.2.0.h5")

In [14]:
model.save("ML-DAN_v.2.0.keras")

In [15]:
model.save_weights('model_weights.weights.h5')

In [19]:
import soundfile as sf
import os

def load_audio_for_test(file_path, target_sr=SAMPLE_RATE, segment_length=SAMPLE_RATE * 5):
    """
    Завантажує аудіофайл і повертає його сегменти потрібної довжини для подальшої обробки.
    """
    audio, sr = librosa.load(file_path, sr=target_sr)
    audio_segments = []

    for start in range(0, len(audio), segment_length):
        segment = audio[start:start + segment_length]
        if len(segment) < segment_length:
            segment = np.pad(segment, (0, segment_length - len(segment)))
        audio_segments.append(segment.reshape(1, -1, 1))

    return audio_segments


def denoise_audio(model, input_file):
    audio_segments = load_audio_for_test(input_file)
    denoised_audio = []

    for segment in audio_segments:
        denoised_segment = model.predict(segment).squeeze()
        denoised_audio.append(denoised_segment)

    denoised_audio = np.concatenate(denoised_audio)
    output_path = os.path.join(os.path.dirname(input_file), "denoised_" + os.path.basename(input_file).split('.')[0] + ".wav")
    sf.write(output_path, denoised_audio, SAMPLE_RATE)
    print(f"Очищене аудіо збережено у файлі: {output_path}")


In [20]:
denoise_audio(model, '../../data/audios/english/train/blended/19-198-0002.mp3')

1/1 ━━━━━━━━━━━━━━━━━━━━ 3s 3s/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 1s/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 1s/step
Очищене аудіо збережено у файлі: ../../data/audios/english/train/blended\denoised_19-198-0002.wav
